In [2]:
from transformers import BertTokenizer, TFBertModel, TFBertForSequenceClassification
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset
import tensorflow as tf 
import numpy as np
from sklearn.preprocessing import LabelEncoder
import multiprocessing
import pickle

In [3]:
df = pd.read_csv("datasets/combined_bills.csv")
df = df.dropna()


In [ ]:
label_encoder = LabelEncoder()
label_encoder.fit(df['Minor'])
df['Minor_Encoded'] = label_encoder.transform(df['Minor'])

with open("label_encoder.pkl", "wb") as f:
    pickle.dump(label_encoder, f)




: 

In [4]:
x = label_encoder.inverse_transform([5])
x

array([107.])

In [5]:
df = df[["Title", "Minor_Encoded"]]

In [6]:
df.rename(columns={"Title": "title", "Minor_Encoded": "minor_encoded"}, inplace=True)

# Convert the DataFrame to a list of dictionaries
data = df.to_dict(orient="records")

C:\Users\imaad\AppData\Local\Temp\ipykernel_1092\3411797025.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={"Title": "title", "Minor_Encoded": "minor_encoded"}, inplace=True)


In [7]:
dataset = Dataset.from_list(data)
dataset = dataset.train_test_split(test_size=0.2)

dataset

DatasetDict({
    train: Dataset({
        features: ['title', 'minor_encoded'],
        num_rows: 373426
    })
    test: Dataset({
        features: ['title', 'minor_encoded'],
        num_rows: 93357
    })
})

In [8]:
bert_model = TFBertModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [9]:

def tokenize(data):
    return tokenizer(data["title"], padding="max_length", truncation=True )

num_cores = multiprocessing.cpu_count()

tokenized_dataset = dataset.map(tokenize, batched=True, batch_size=16,  load_from_cache_file=True)


Map: 100%|██████████| 93357/93357 [01:16<00:00, 1214.61 examples/s]


In [10]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['title', 'minor_encoded', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 373426
    })
    test: Dataset({
        features: ['title', 'minor_encoded', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 93357
    })
})

In [11]:
# setting 'input_ids', 'attention_mask', 'token_type_ids', and 'label'
# to the tensorflow format. Now if you access this dataset you will get these
# columns in `tf.Tensor` format

tokenized_dataset.set_format('tf', 
                            columns=['input_ids', 'attention_mask', 'token_type_ids', 'minor_encoded'])

# setting BATCH_SIZE to 64.
BATCH_SIZE = 64

def order(inp):
    '''
    This function will group all the inputs of BERT
    into a single dictionary and then output it with
    labels.
    '''
    data = list(inp.values())
    return {
        'input_ids': data[1],
        'attention_mask': data[2],
        'token_type_ids': data[3]
    }, data[0]

# converting train split of `emotions_encoded` to tensorflow format
train_dataset = tf.data.Dataset.from_tensor_slices(tokenized_dataset['train'][:])
# set batch_size and shuffle
train_dataset = train_dataset.batch(BATCH_SIZE).shuffle(1000)
# map the `order` function
train_dataset = train_dataset.map(order, num_parallel_calls=tf.data.AUTOTUNE)

# ... doing the same for test set ...
test_dataset = tf.data.Dataset.from_tensor_slices(tokenized_dataset['test'][:])
test_dataset = test_dataset.batch(BATCH_SIZE)
test_dataset = test_dataset.map(order, num_parallel_calls=tf.data.AUTOTUNE)

In [12]:
class BERTForClassification(tf.keras.Model):

    def __init__(self, bert_model, num_classes):
        super().__init__()
        self.bert = bert_model
        self.fc = tf.keras.layers.Dense(num_classes, activation='softmax')

    def call(self, inputs):
        x = self.bert(inputs)[1]
        return self.fc(x)
    

In [13]:
classifier  = BERTForClassification(bert_model, num_classes=232)

classifier.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics = ['accuracy']
)

2.18.0


In [10]:
print("GPUs Available:", tf.config.list_physical_devices('GPU'))


GPUs Available: []


In [ ]:
history =  classifier.fit(train_dataset, epochs=3 )

Epoch 1/3


   4/5835 [..............................] - ETA: 627:56:07 - loss: 5.6706 - accuracy: 0.0000e+00